In [ ]:
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as mp
import matplotlib as mpl
import geopandas as gpd
from matplotlib_scalebar.scalebar import ScaleBar
import contextily as ctx
# import geovoronoi
from geovoronoi import voronoi_regions_from_coords
from geovoronoi.plotting import subplot_for_map, plot_voronoi_polys_with_points_in_area
from shapely.geometry import MultiPoint, Point

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
from shapely.ops import voronoi_diagram

## Read D3D results (entire grid)

In [ ]:
# filenames = ['morpho_d3d_output.p', 'NOmorpho_d3d_output.p']
filename = 'morpho_d3d_output.p'
file = open(r'../data/%s'%filename, 'rb')
mat_contents = pickle.load(file) #load from pickle file

#### Norm vals for matching colormaps in plots

In [ ]:
def norm_vals(array, vmin=None, vmax=None):
    if all(v is not None for v in [vmin, vmax]):
        return (array - vmin)/(vmax - vmin)
    else:
        return (array - min(array))/(max(array) - min(array))

### create voronoi polygons for region

https://shapely.readthedocs.io/en/stable/manual.html#voronoi-diagram

In [ ]:
# x_bounds = [547800, 548100]
# y_bounds = [5417650,5418350]
subset = True
if subset == True:
    #bounds corresponding to qgis inset
    x_bounds = [547000, 549500]
    y_bounds = [5416200,5419000]    
    #wider region that includes 'avulsion/new' channel
    # x_bounds = [547450, 548400]
    # y_bounds = [5417350,5418750]
    #region that matches flood arrows
    # x_bounds = [5417650,5418350]
    # y_bounds = [547800, 548100]
if subset == False: #SUPER SLOW FOR ENTIER DOMAIN!!!
    x_bounds = [mat_contents['G']['face']['FlowElem_x'].min(), mat_contents['G']['face']['FlowElem_x'].max()]
    y_bounds = [mat_contents['G']['face']['FlowElem_y'].min(), mat_contents['G']['face']['FlowElem_y'].max()]

spatial_filter = ((mat_contents['G']['face']['FlowElem_x'] >= x_bounds[0]) & \
                  (mat_contents['G']['face']['FlowElem_x'] <= x_bounds[1]) & \
                  (mat_contents['G']['face']['FlowElem_y'] >= y_bounds[0]) & \
                  (mat_contents['G']['face']['FlowElem_y'] <= y_bounds[1])
                 )

In [ ]:
d3d_coords = np.array([(x,y) for x, y in zip(mat_contents['G']['face']['FlowElem_x'][spatial_filter],
                                    mat_contents['G']['face']['FlowElem_y'][spatial_filter])])
mtp_d3d_coords = MultiPoint(d3d_coords)

In [ ]:
if subset == True:
    polys, pts = voronoi_regions_from_coords(d3d_coords, mtp_d3d_coords.convex_hull)
if subset == False:
    polys, pts = voronoi_regions_from_coords(d3d_coords, d3d_poly)
# voronoi_regions = voronoi_diagram(mtp_d3d_coords, envelope=mtp_d3d_coords.convex_hull) #doesn't organize polygons!!!

### Plot initial & Final elv

In [ ]:
ordered_polys = [polys[i] for i in range(len(polys))]
ordered_centers = [Point(d3d_coords[pts[i]][0]) for i in range(len(polys))]

Zt0 = mat_contents['D'][0][0]['face']['z_cc'][spatial_filter]
ordered_Zt0 = [Zt0[pts[i]][0] for i in range(len(polys))]

Ztend = mat_contents['D'][450][0]['face']['z_cc'][spatial_filter]
ordered_Ztend = [Ztend[pts[i]][0] for i in range(len(polys))]

dfZ = pd.DataFrame()
dfZ['polys'] = ordered_polys
dfZ['starting_elev'] = ordered_Zt0
dfZ['ending_elev'] = ordered_Ztend

gdfZ = gpd.GeoDataFrame(dfZ, crs=32610, geometry=dfZ.polys)
poly_buffer = 40

#just bar and avulsion channel
# vlims = (20, 30)
# cmap = 'cividis'
#for qgis
vlims = (22.8, 31.4)
cmap='magma_r'

f, (ax, ax1) = mp.subplots(ncols=2, figsize=(12,5))
gdfZ.plot(column='starting_elev', cmap=cmap, ax=ax, legend=True, vmin=vlims[0], vmax=vlims[1], legend_kwds={'label': 'elevation (m)'})
# nooksack_poly.buffer(poly_buffer).plot(
#     ax=ax, zorder=1, color='none', edgecolor='k',
#     linewidth=1.5, linestyle='--', alpha=0.75)

ax.add_artist(ScaleBar(1))
ax.axis('off')
ax.set_title('initial elevation')


gdfZ.plot(column='ending_elev', cmap=cmap, ax=ax1, legend=True, vmin=vlims[0], vmax=vlims[1], legend_kwds={'label': 'elevation (m)'})
# nooksack_poly.buffer(poly_buffer).plot(
#     ax=ax1, zorder=1, color='none', edgecolor='k',
#     linewidth=1.5, linestyle='--', alpha=0.75)

ax1.add_artist(ScaleBar(1))
ax1.axis('off')
ax1.set_title('final elevation')
#plot without avulsion channel
# ax.set_ylim(5417400,5418500)
# ax.set_xlim(547600, 548255)
# ax1.set_ylim(5417400,5418500)
# ax1.set_xlim(547600, 548255)
#blot using bounds of filter
ax.set_ylim(y_bounds[0], y_bounds[1])
ax.set_xlim(x_bounds[0], x_bounds[1])
ax1.set_ylim(y_bounds[0], y_bounds[1])
ax1.set_xlim(x_bounds[0], x_bounds[1])
mp.subplots_adjust(wspace=0.05)

# mp.savefig(r'..\figs\model_output\general_output\morpho_on\starting_and_ending_elev_new_channels.png',
#            dpi=200,
#            bbox_inches='tight')  